In [1]:
import pandas as pd
import gzip
import numpy as np
import time
import os
import re
import progressbar

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.insert(0, '/media/andrii/earth/Katia/CS_MasterThesis/Red_links_Project_for_Wiki_draft/py')
import utils

### Load en_wiki

In [4]:
#path to data
PATH_TO_DATA = '/media/andrii/earth/Katia/CS_MasterThesis/data/'
PATH_TO_DATA_UK = PATH_TO_DATA+"ukwiki/"
PATH_TO_DATA_EN = PATH_TO_DATA+"enwiki/en_parsed/"

In [5]:
data_time = '20180920'

In [9]:
# get list of archives to unpack for uk wiki
ENWIKI_ART_FNMS = []
for file in os.listdir(PATH_TO_DATA_EN):
    if re.match(r"enwiki-20180920-pages-meta-current\d+\.xml-p\d+p\d+\.bz2_art\.csv.gz", file):
        ENWIKI_ART_FNMS.append(file) 
 

In [10]:
len(ENWIKI_ART_FNMS)

55

In [11]:
EN_ID_NAME = "enwiki-"+data_time+"-id_name.csv"

In [12]:
def unpack(file_name):
    file_name_new = file_name.replace(".gz","")
    with gzip.open(file_name, 'rb') as f_in, open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    return file_name_new

In [13]:
start_time = time.time()
# loading all articles to dataframe
df_en = pd.DataFrame()
n_files = len(ENWIKI_ART_FNMS)
print(n_files)

for index in range(n_files):
    print(str(index+1) + '/' + str(n_files))
    fn = ENWIKI_ART_FNMS[index]
    fn = PATH_TO_DATA_EN+fn
    print(fn)
    fn_new = unpack(fn)
    df_articles = pd.read_csv(fn_new, encoding='UTF-8', sep='^')  
    df_articles = df_articles[df_articles['is_red_link']==False][['id','link_id']]
    df_en = pd.concat((df_en, df_articles))
    print("df_articles size: {}. df_articles columns: {}\n".format(df_articles.shape, list(df_articles.columns)))
    os.remove(fn_new)
    
print('Total time: %.1f minutes' % ((time.time() - start_time)/60))

55
1/55
/media/andrii/earth/Katia/CS_MasterThesis/data/enwiki/en_parsed/enwiki-20180920-pages-meta-current14.xml-p6197598p7697598.bz2_art.csv.gz
df_articles size: (4553133, 2). df_articles columns: ['id', 'link_id']

2/55
/media/andrii/earth/Katia/CS_MasterThesis/data/enwiki/en_parsed/enwiki-20180920-pages-meta-current27.xml-p54663462p56163462.bz2_art.csv.gz
df_articles size: (2318617, 2). df_articles columns: ['id', 'link_id']

3/55
/media/andrii/earth/Katia/CS_MasterThesis/data/enwiki/en_parsed/enwiki-20180920-pages-meta-current27.xml-p53163462p54663462.bz2_art.csv.gz
df_articles size: (2611380, 2). df_articles columns: ['id', 'link_id']

4/55
/media/andrii/earth/Katia/CS_MasterThesis/data/enwiki/en_parsed/enwiki-20180920-pages-meta-current15.xml-p7744803p9244803.bz2_art.csv.gz
df_articles size: (4160343, 2). df_articles columns: ['id', 'link_id']

5/55
/media/andrii/earth/Katia/CS_MasterThesis/data/enwiki/en_parsed/enwiki-20180920-pages-meta-current25.xml-p35452816p36952816.bz2_art.

df_articles size: (2974215, 2). df_articles columns: ['id', 'link_id']

40/55
/media/andrii/earth/Katia/CS_MasterThesis/data/enwiki/en_parsed/enwiki-20180920-pages-meta-current4.xml-p200511p352689.bz2_art.csv.gz
df_articles size: (3563252, 2). df_articles columns: ['id', 'link_id']

41/55
/media/andrii/earth/Katia/CS_MasterThesis/data/enwiki/en_parsed/enwiki-20180920-pages-meta-current23.xml-p26823661p28323661.bz2_art.csv.gz
df_articles size: (3301985, 2). df_articles columns: ['id', 'link_id']

42/55
/media/andrii/earth/Katia/CS_MasterThesis/data/enwiki/en_parsed/enwiki-20180920-pages-meta-current27.xml-p44163462p45663462.bz2_art.csv.gz
df_articles size: (2745988, 2). df_articles columns: ['id', 'link_id']

43/55
/media/andrii/earth/Katia/CS_MasterThesis/data/enwiki/en_parsed/enwiki-20180920-pages-meta-current26.xml-p38067203p39567203.bz2_art.csv.gz
df_articles size: (3070338, 2). df_articles columns: ['id', 'link_id']

44/55
/media/andrii/earth/Katia/CS_MasterThesis/data/enwiki/en_pa

In [14]:
print(df_en.shape)
print(df_en.columns)

(161017377, 2)
Index(['id', 'link_id'], dtype='object')


In [17]:
# article names
df_en_name = pd.read_csv(PATH_TO_DATA_EN+EN_ID_NAME,  encoding='UTF-8', quotechar="\"", sep='^')
df_en_name = df_en_name[['id','title']]
print(df_en_name.shape)
print(df_en_name.columns)

(5719743, 2)
Index(['id', 'title'], dtype='object')


### Load uk_wiki

In [21]:
UKWIKI_ART_FNMS = []
for file in os.listdir(PATH_TO_DATA_UK):
    if re.match(r"ukwiki-20180920-pages-links.csv", file):
        UKWIKI_ART_FNMS.append(file)

In [22]:
UKWIKI_ART_FNMS

['ukwiki-20180920-pages-links.csv']

In [23]:
UK_ID_NAME = "ukwiki-"+data_time+"-id_name.csv"
UK_ID_NAME

'ukwiki-20180920-id_name.csv'

In [24]:
start_time = time.time()

# loading existing articles and red links to dataframes
df_uk_blue = pd.DataFrame()
df_uk_red = pd.DataFrame()
n_files = len(UKWIKI_ART_FNMS)
for index in range(n_files):
    print(str(index+1) + '/' + str(n_files))
    fn = UKWIKI_ART_FNMS[index]
    fn = PATH_TO_DATA_UK+fn
    print(fn)
    df_articles = pd.read_csv(fn)
    
    df_articles_blue = df_articles[df_articles['is_red_link']==False]
    df_uk_blue = pd.concat((df_uk_blue, df_articles_blue[['id','link_id']]))
    df_articles_red = df_articles[df_articles['is_red_link']]
    df_uk_red = pd.concat((df_uk_red, df_articles_red[['id','link_val']]))
    
    print("df_articles size: {}. df_articles columns: {}\n".format(df_articles.shape, list(df_articles.columns)))

print('Total time: %.1f minutes' % ((time.time() - start_time)/60))

1/1
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/ukwiki-20180920-pages-links.csv
df_articles size: (22693776, 4). df_articles columns: ['id', 'link_id', 'link_val', 'is_red_link']

Total time: 0.3 minutes


In [25]:
print(df_uk_blue.shape)
print(df_uk_blue.columns)

print(df_uk_red.shape)
print(df_uk_red.columns)

(19818822, 2)
Index(['id', 'link_id'], dtype='object')
(2874954, 2)
Index(['id', 'link_val'], dtype='object')


In [26]:
# article names
df_uk_name = pd.read_csv(PATH_TO_DATA_UK+UK_ID_NAME, sep='^')

print(df_uk_name.shape)
print(df_uk_name.columns)

(819212, 3)
Index(['id', 'title', 'length'], dtype='object')


In [34]:
# # add ids for red links

# # find maximum id among blue links
max_id = np.max(df_uk_name['id'])

# # find unique titles among red links, create df with
# red_links = df_uk_red['link_val'].unique()
# red_links_ids = pd.DataFrame({'title': red_links, 
#                               'id': np.arange(max_id+1, len(red_links)+max_id+1)})
# red_links_ids['is_red_link'] = True

# # create dataframe with titles and red link tickets / add it to article titles df
# df_uk_name['is_red_link'] = False
# df_uk_name = pd.concat((df_uk_name,red_links_ids))
# df_uk_name = df_uk_name.reset_index(drop=True)

# print(df_uk_name.shape)
# print(df_uk_name.columns)


In [27]:
# # add link ids to df_uk_red
# df_uk_red = df_uk_red.merge(right = red_links_ids, left_on = 'link_val', right_on = 'title', how = 'left')
# df_uk_red = df_uk_red[['id_x','id_y']]
# df_uk_red.columns = ['id', 'link_id']

df_uk_red = pd.read_csv(PATH_TO_DATA_UK+'all_uk_redlinks_with_parent_ids.csv')

In [28]:
df_uk_red.head()

,id_parent,red_link_name,red_link_id
0,13,Географія розвитку,2817993
1,13,Географія часу,2817994
2,1928599,Географія часу,2817994
3,13,Туристична географія,2817995
4,584,рентгенівська спектроскопія поглинання,2817996


In [29]:
df_uk_red = df_uk_red[['id_parent', 'red_link_id']]
df_uk_red.columns = ['id', 'link_id']

In [30]:
# concatenate red and blue article to one df
df_uk = pd.concat((df_uk_red, df_uk_blue))

In [31]:
print(df_uk.shape)
print(df_uk_blue.shape[0] + df_uk_red.shape[0])

(22691094, 2)
22691094


In [32]:
del df_uk_red, df_uk_blue

### Load uk-en correpondences

In [35]:
fn = PATH_TO_DATA_UK+'20180920-langlinks_uk_en.csv'
df_link = pd.read_csv(fn, encoding='UTF-8', quotechar='\'')
print(df_link.shape)
print(df_link.columns)

(599612, 3)
Index(['ll_from', 'll_lang', 'll_title'], dtype='object')


In [36]:
df_link.head()

,ll_from,ll_lang,ll_title
0,2112530,en,! (The Dismemberment Plan album)
1,1366003,en,! (disambiguation)
2,2141483,en,!!
3,425480,en,!!!
4,848226,en,!Action Pact!


In [ ]:
# # remove values links to negative values
# df_link = df_link[(df_link['id_en']>0) & (df_link['id_en']!=49244)]
# print(df_link.shape)

In [37]:
df_en_name.head()

,id,title
0,6197600,Flying Tigers (film)
1,6197604,Mariano Andreu
2,6197607,David Prain
3,6197617,Rut Brandt
4,6197621,Gregory C. Coleman


In [38]:
# add uk correspondences to en article names
df_en_name = pd.merge(df_en_name, df_link, left_on = 'title', right_on = 'll_title', how = 'left')

In [39]:
df_en_name.head()

,id,title,ll_from,ll_lang,ll_title
0,6197600,Flying Tigers (film),NaN,NaN,NaN
1,6197604,Mariano Andreu,NaN,NaN,NaN
2,6197607,David Prain,1892860.0,en,David Prain
3,6197617,Rut Brandt,NaN,NaN,NaN
4,6197621,Gregory C. Coleman,NaN,NaN,NaN


In [40]:
df_en_name_ = df_en_name[['id','title', 'll_from']]
df_en_name_.columns = ['id', 'title', 'id_uk']

In [41]:
df_en_name_.head()

,id,title,id_uk
0,6197600,Flying Tigers (film),NaN
1,6197604,Mariano Andreu,NaN
2,6197607,David Prain,1892860.0
3,6197617,Rut Brandt,NaN
4,6197621,Gregory C. Coleman,NaN


In [42]:
# several en articles have multiple corresponding uk acticles, just remove oldest one uk article
df_en_translated = (df_en_name_[~df_en_name_['id_uk'].isnull()])[['id', 'id_uk']]
df_en_translated.columns = ['id_en','id_uk']
df_en_translated = df_en_translated.sort_values(by = 'id_en')
df_en_translated = df_en_translated.drop_duplicates(keep = 'first', subset = ['id_uk'])

In [43]:
# add en correspondences to uk article names

In [45]:
df_articles.head()

,id,link_id,link_val,is_red_link
0,13,884280,Об'єкт (філософія),False
1,13,21623,Ератосфен,False
2,13,274944,географ,False
3,13,26194,Клавдій Птолемей,False
4,13,20459,Відродження,False


In [46]:
df_uk_name = df_articles.merge(right = df_en_translated, left_on = 'id', right_on = 'id_uk', how = 'left')

In [47]:
df_uk_name.head()

,id,link_id,link_val,is_red_link,id_en,id_uk
0,13,884280,Об'єкт (філософія),False,18963910.0,13.0
1,13,21623,Ератосфен,False,18963910.0,13.0
2,13,274944,географ,False,18963910.0,13.0
3,13,26194,Клавдій Птолемей,False,18963910.0,13.0
4,13,20459,Відродження,False,18963910.0,13.0


In [48]:
df_uk_name = df_uk_name[['id','link_val','is_red_link','id_en']]
df_uk_name.columns = ['id','title','is_red_link','id_en']
df_uk_name = df_uk_name.sort_values(by = 'id')

In [49]:
df_uk_name.head()

,id,title,is_red_link,id_en
0,13,Об'єкт (філософія),False,18963910.0
59,13,Карл Ріттер,False,18963910.0
58,13,Александер фон Гумбольдт,False,18963910.0
57,13,Герард Меркатор,False,18963910.0
56,13,Мухаммад Аль-Ідрісі,False,18963910.0


In [50]:
df_en_name_.head()

,id,title,id_uk
0,6197600,Flying Tigers (film),NaN
1,6197604,Mariano Andreu,NaN
2,6197607,David Prain,1892860.0
3,6197617,Rut Brandt,NaN
4,6197621,Gregory C. Coleman,NaN


In [53]:
# save names to files
df_uk_name.to_csv(PATH_TO_DATA_UK+'uk_names.csv', header=True, index=False, sep='^')

In [54]:
df_en_name_.to_csv(PATH_TO_DATA_EN+'en_names.csv', header=True, index=False, sep='^')

In [55]:
df_uk_name.head()

,id,title,is_red_link,id_en
0,13,Об'єкт (філософія),False,18963910.0
59,13,Карл Ріттер,False,18963910.0
58,13,Александер фон Гумбольдт,False,18963910.0
57,13,Герард Меркатор,False,18963910.0
56,13,Мухаммад Аль-Ідрісі,False,18963910.0


In [6]:
# load names 
df_uk_name = pd.read_csv(PATH_TO_DATA_UK+'uk_names.csv', sep='^')
df_en_name = pd.read_csv(PATH_TO_DATA_EN+'en_names.csv', sep='^')

### Encode en nontranslated article by its incoming links

In [7]:
# select non-translated articles
en_nontranslated = np.array(df_en_name[df_en_name['id_uk'].isnull()]['id'])
print('Nontranslated en acticles: %6d' % (len(en_nontranslated)))

Nontranslated en acticles: 5264607


In [15]:
# select non-translated acticles that have at least 20 distinct incoming links
en_nontranslated = df_en[df_en['link_id'].isin(en_nontranslated)].groupby('link_id') \
            .agg({'id': lambda x: x.nunique()})
en_nontranslated = en_nontranslated.reset_index()
en_nontranslated.columns = ['id','n_incoming']
en_nontranslated = en_nontranslated[en_nontranslated['n_incoming']>=20]
en_nontranslated = np.array(en_nontranslated['id'])
en_nontranslated_ids = np.sort(en_nontranslated)
print('Nontranslated en acticles with at least 20 incoming links: %6d' % (len(en_nontranslated)))

Nontranslated en acticles with at least 20 incoming links: 665807


In [16]:
# encode each nontranslated article by its incoming links
en_nontranslated_encoding_df = df_en[df_en['link_id'].isin(en_nontranslated)]
en_nontranslated_encoding_df = en_nontranslated_encoding_df.sort_values(by = ['link_id','id'])
en_nontranslated_encoding_df = en_nontranslated_encoding_df.reset_index(drop = True)
indices = np.array(en_nontranslated_encoding_df.drop_duplicates(keep='first', subset=['link_id']).index)

en_nontranslated_encoding = []
pbar = progressbar.ProgressBar()
for i in pbar(range(0, len(indices))):
    if i == len(indices) - 1:
        this_encoding = set(en_nontranslated_encoding_df.iloc[indices[i]:]['id'])
    else: 
        this_encoding = set(en_nontranslated_encoding_df.iloc[indices[i]:indices[i+1]]['id'])
    en_nontranslated_encoding.append(this_encoding)
    
del en_nontranslated_encoding_df, en_nontranslated

100% |########################################################################|


In [17]:
# save encoding to file
np.save(PATH_TO_DATA_EN+'en_nontranslated_encoding.npy', arr=en_nontranslated_encoding)
np.save(PATH_TO_DATA_EN+'en_nontranslated_ids.npy', arr=en_nontranslated_ids)

In [19]:
# load encoding from file
en_nontranslated_encoding = np.load(PATH_TO_DATA_EN+'en_nontranslated_encoding.npy', allow_pickle=True)
en_nontranslated_ids = np.load(PATH_TO_DATA_EN+'en_nontranslated_ids.npy', allow_pickle=True)

### Encode red links incoming en links

In [35]:
# select only red links from uk_wiki
df_uk_red = df_uk[df_uk['link_id']>max_id]
uk_red_ids = df_uk_red['link_id'].unique()
print('Number of red links in uk wiki: %d' %(len(uk_red_ids)))

Number of red links in uk wiki: 1554986


In [36]:
df_uk_red = df_uk_red.drop_duplicates()

In [37]:
len(df_uk_red)

2443148

In [38]:
df_uk_red.head()

,id,link_id
0,13,2817993
1,13,2817994
2,1928599,2817994
3,13,2817995
4,584,2817996


In [39]:
df_uk_name.head()

,id,title,length
0,3,Головна сторінка,3197
1,13,Географія,16843
2,584,Атом,35112
3,585,Мільярд,1759
4,586,Ядро,8064


In [41]:
ukredlinks_with_enids_sample = pd.read_csv(PATH_TO_DATA_EN+'ukredlinks_with_enids_sample.csv', sep='^')

In [42]:
ukredlinks_with_enids_sample.head()

,red_link_name,id_parent,red_link_id,id,title
0,Каунт Бейсі,625,2818114,34651.0,1904
1,Каунт Бейсі,36823,2818114,605856.0,WordPress
2,Каунт Бейсі,202525,2818114,50350.0,Ella Fitzgerald
3,Каунт Бейсі,202608,2818114,22519.0,Oscar Peterson
4,Каунт Бейсі,202613,2818114,2193014.0,Lester Young


In [43]:
# for every red link from the sample find its correponding en incoming link (only distinct)
df_uk_red_encoding = ukredlinks_with_enids_sample[['id', 'red_link_id']]
df_uk_red_encoding.columns = ['id_en','link_id']
df_uk_red_encoding = df_uk_red_encoding[~df_uk_red_encoding['id_en'].isnull()]
df_uk_red_encoding = df_uk_red_encoding.drop_duplicates()

In [44]:
df_uk_red_encoding.shape

(164752, 2)

In [45]:
df_uk_red_encoding.head()

,id_en,link_id
0,34651.0,2818114
1,605856.0,2818114
2,50350.0,2818114
3,22519.0,2818114
4,2193014.0,2818114


In [46]:
# for every red link from the sample calculate number of incoming links
df_uk_red_incoming = df_uk_red_encoding['link_id'].value_counts().reset_index()
df_uk_red_incoming.columns = ['id','n_incoming']

In [47]:
# select only those red links that can be encoded with at least 20 en incoming links
uk_red_ids = np.array(df_uk_red_incoming[df_uk_red_incoming['n_incoming']>=20]['id'])
print('Number of red links with at least 20 distinct incoming en links: %d' % (len(uk_red_ids)))

Number of red links with at least 20 distinct incoming en links: 3198


In [48]:
# encode top red links using incoming en links
uk_red_ids = np.sort(uk_red_ids)
df_uk_red_encoding = df_uk_red_encoding[df_uk_red_encoding['link_id'].isin(uk_red_ids)]
df_uk_red_encoding = df_uk_red_encoding.sort_values(by = 'link_id').reset_index(drop = True)
indices = np.array(df_uk_red_encoding.drop_duplicates(keep='first', subset=['link_id']).index)

uk_red_encoding = []
pbar = progressbar.ProgressBar()
for i in pbar(range(0, len(indices))):
    if i == len(indices)-1:
        this_encoding = set(df_uk_red_encoding.iloc[indices[i]:]['id_en'])
    else:
        this_encoding = set(df_uk_red_encoding.iloc[indices[i]:indices[i+1]]['id_en'])
    uk_red_encoding.append(this_encoding)

100% |########################################################################|


In [49]:
del df_uk_red_incoming, df_uk_red_encoding

In [50]:
# save encoding to file
np.save(file='uk_red_ids.npy', arr=uk_red_ids)
np.save(file='uk_red_encoding.npy', arr=uk_red_encoding)

In [52]:
# load encoding from file
uk_red_ids = np.load('uk_red_ids.npy', allow_pickle=True)
uk_red_encoding = np.load('uk_red_encoding.npy', allow_pickle=True)

In [53]:
df_uk_name.head()

,id,title,length
0,3,Головна сторінка,3197
1,13,Географія,16843
2,584,Атом,35112
3,585,Мільярд,1759
4,586,Ядро,8064


In [55]:
print(len(list(df_uk_name[df_uk_name['id'].isin(uk_red_ids)]['title'])))

0


In [58]:
red_links_names = ukredlinks_with_enids_sample[ukredlinks_with_enids_sample['red_link_id'].isin(uk_red_ids)]['red_link_name'].to_frame()
red_links_names = red_links_names.drop_duplicates()
list_of_red_link = red_links_names['red_link_name'].tolist()

In [59]:
# create df with incoming uk ids and names for every red link
incoming_uk_link_names = []
n_incoming_uk = []
incoming_en_link_names = []
n_incoming_en = []

pbar = progressbar.ProgressBar()
for i in pbar(range(len(uk_red_ids))):
    red_id = uk_red_ids[i]
    incoming_ids = list(df_uk_red[df_uk_red['link_id'] == red_id]['id'])
    incoming_names = set(df_uk_name[df_uk_name['id'].isin(incoming_ids)]['title'])
    incoming_en_names = set(df_en_name[df_en_name['id'].isin(uk_red_encoding[i])]['title'])
              
    incoming_uk_link_names.append(incoming_names)
    n_incoming_uk.append(len(incoming_names))
    incoming_en_link_names.append(incoming_en_names) 
    n_incoming_en.append(len(incoming_en_names))

100% |########################################################################|


In [60]:
df_red_links = pd.DataFrame({'red_link_id': uk_red_ids, 
                          'red_link_name': list_of_red_link,
                          'incoming_uk_names': incoming_uk_link_names,
                          'n_incoming_uk': n_incoming_uk,
                          'incoming_en_names': incoming_en_link_names,
                          'n_incoming_en': n_incoming_en
                          })
              
df_red_links = df_red_links[['red_link_id','red_link_name','incoming_uk_names',
                             'n_incoming_uk', 'incoming_en_names', 'n_incoming_en']]

In [61]:
print(df_red_links.shape)
df_red_links.head(100)

(3198, 6)


,red_link_id,red_link_name,incoming_uk_names,n_incoming_uk,incoming_en_names,n_incoming_en
0,2818114,Каунт Бейсі,"{Мельба Лістон, Джо Ньюмен, Тед Джонс, Бадді К...",45,"{Cecil Payne, Harold Ashby, Lorez Alexandria, ...",42
1,2818231,аль-Каїм,"{1050, 1061, 1066, 1074, 1065, 1054, 1046, 103...",46,"{1050, 1061, 1066, 1074, 1065, 1054, 1046, 107...",46
2,2818265,королева,"{Жанна III (королева Наварри), 1926, 1 серпня,...",57,"{The Twins Effect II, Margaret of Anjou, 1926,...",50
3,2821142,Prentice Hall,"{Транспорт Ватикану, Транспорт Американських В...",253,"{Transport in Ireland, Transport in Guyana, Tr...",231
4,2824527,Tokar.ua,"{Сонячна енергетика України, Російсько-українс...",44,"{Solar power, Yandex.Direct, Popcorn Time, Cal...",37
5,2824727,Monthly Notices of the Royal Astronomical Society,"{VIRGOHI21, Зоря Тігардена, Зони H II, Кентавр...",57,"{Atmosphere of Uranus, VIRGOHI21, Pisces Dwarf...",57
6,2825668,Світ (видавництво),"{Транспорт Судану, Транспорт Північної Кореї, ...",517,"{Transport in Ireland, Transport in Russia, Tr...",472
7,2825773,Дубович Іван Андрійович,"{Транспорт Судану, Транспорт Північної Кореї, ...",487,"{Transport in Ireland, Transport in Russia, Tr...",448
8,2826419,помірні повітряні маси,"{Японське море, Гудзонова затока, Ірландське м...",67,"{Irminger Sea, Foxe Basin, Geographical zone, ...",65
9,2826499,Доповідь про торгівлю людьми,"{Населення Катару, Населення Тринідаду і Тобаг...",206,"{Demographics of Greece, Demographics of Bosni...",200


In [62]:
# save red links incoming links to files
df_red_links.to_csv(PATH_TO_DATA_UK+'red_links_summary_jaccard_model.csv.gz', compression='gzip', header=True, index=False)

In [63]:
# load 
df_red_links = pd.read_csv(PATH_TO_DATA_UK+'red_links_summary_jaccard_model.csv.gz')

### Find most similar uk articles for red links

In [64]:
# jaccard similarity
def jaccard(a, b):
    c = a.intersection(b)
    return len(c) / (len(a) + len(b) - len(c))

In [124]:
! Version 1

en_article_list = []
en_article_found = []

pbar = progressbar.ProgressBar()
for j in pbar(range(len(uk_red_ids))):
    red_link = uk_red_ids[j]
    red_link_encoding = uk_red_encoding[j]
    similarities = []
    for i in range(len(en_nontranslated_ids)):
        similarity = jaccard(red_link_encoding, en_nontranslated_encoding[i])
        similarities.append(similarity)
    
    # select top 5 most similar uk article, zero similarities excluded
    similarities = np.array(similarities)
    indices = np.argsort(-similarities)[:5]
    score = np.round(similarities[indices],3)
    n_nonzero = np.sum(score>0)
    indices = np.sort(indices[:n_nonzero])

    # scores and article names sorted by article ids
    score = np.round(similarities[indices],3)
    en_article_ids = en_nontranslated_ids[indices]
    en_article_names = list(df_en_name[df_en_name['id'].isin(en_article_ids)]['title'])
    
    # sort by jaccard similarity
    d = dict(zip(en_article_names, score))
    d_sorted = sorted(((value, key) for (key,value) in d.items()), reverse=True)
    
    # save to list
    en_article_list.append(d_sorted)
    
    if len(en_article_names)>0:
        en_article_found.append(True)
    else:
        en_article_found.append(False)

        
# create df to show results
df_similarities = pd.DataFrame({'red_link_id': uk_red_ids,
                   'en_similar_found': en_article_found, 
                   'en_similar': en_article_list
                  })
    
    

100% |########################################################################|


In [65]:
# version for getting all candidates


en_article_list = []
en_article_found = []

pbar = progressbar.ProgressBar()
for j in pbar(range(len(uk_red_ids))):
    red_link = uk_red_ids[j]
    red_link_encoding = uk_red_encoding[j]
    similarities = []
    for i in range(len(en_nontranslated_ids)):
        similarity = jaccard(red_link_encoding, en_nontranslated_encoding[i])
        similarities.append(similarity)
    
    # zero similarities excluded
    similarities = np.array(similarities)
    indices = np.argsort(-similarities)
    score = np.round(similarities[indices],3)
    n_nonzero = np.sum(score>0)
    indices = np.sort(indices[:n_nonzero])

    # scores and article names sorted by article ids
    score = np.round(similarities[indices],3)
    en_article_ids = en_nontranslated_ids[indices]
    en_article_names = list(df_en_name[df_en_name['id'].isin(en_article_ids)]['title'])
    
    # sort by jaccard similarity
    d = dict(zip(en_article_names, score))
    d_sorted = sorted(((value, key) for (key,value) in d.items()), reverse=True)
    
    # save to list
    en_article_list.append(d_sorted)
    
    if len(en_article_names)>0:
        en_article_found.append(True)
    else:
        en_article_found.append(False)

        
# create df to show results
df_similarities = pd.DataFrame({'red_link_id': uk_red_ids,
                   'en_similar_found': en_article_found, 
                   'en_similar': en_article_list
                  })
    
    

100% |########################################################################|


In [74]:
df_red_results = df_red_links.merge(right=df_similarities, right_on = 'red_link_id',
                                     left_on = 'red_link_id', how = 'left')

In [75]:
df_red_results.head()

,red_link_id,red_link_name,incoming_uk_names,n_incoming_uk,incoming_en_names,n_incoming_en,en_similar_found,en_similar
0,2818114,Каунт Бейсі,"{'Мельба Лістон', 'Джо Ньюмен', 'Тед Джонс', '...",45,"{'Cecil Payne', 'Harold Ashby', 'Lorez Alexand...",42,True,"[(0.081, James Chirillo), (0.076, Bud Freeman)..."
1,2818231,аль-Каїм,"{'1050', '1061', '1066', '1074', '1065', '1054...",46,"{'1050', '1061', '1066', '1074', '1065', '1054...",46,True,"[(0.031, Anselm of Lucca), (0.03, Kilbeggan), ..."
2,2818265,королева,"{'Жанна III (королева Наварри)', '1926', '1 се...",57,"{'The Twins Effect II', 'Margaret of Anjou', '...",50,True,"[(0.033, Charles Weeghman), (0.029, Thomas Bol..."
3,2821142,Prentice Hall,"{'Транспорт Ватикану', 'Транспорт Американськи...",253,"{'Transport in Ireland', 'Transport in Guyana'...",231,True,"[(0.078, Skyways Limited), (0.067, Connexxion)..."
4,2824527,Tokar.ua,"{'Сонячна енергетика України', 'Російсько-укра...",44,"{'Solar power', 'Yandex.Direct', 'Popcorn Time...",37,True,"[(0.05, Turbo-diesel), (0.032, List of virtual..."


In [76]:
df_red_results['en_similar_found'].value_counts()

True     3187
False      11
Name: en_similar_found, dtype: int64

In [77]:
# save results with all candidates to file
df_red_results.to_csv(PATH_TO_DATA_UK+'uk_red_links_results_all_candidates.csv', header=True, index=False, encoding = 'UTF-8', sep='^')

In [78]:
# save results to file
df_red_results.to_csv(PATH_TO_DATA_UK+'uk_red_links_results.csv', header=True, index=False, encoding = 'UTF-8', sep='^')

In [79]:
# load 
df_red_results = pd.read_csv(PATH_TO_DATA_UK+'uk_red_links_results.csv', encoding = 'UTF-8', sep='^')

In [80]:
df_red_results = df_red_results[['red_link_name', 'en_similar']]

In [81]:
# save results to file
df_red_results.to_csv(PATH_TO_DATA_UK+'uk_red_links_results_lighter.csv', header=True, index=False, encoding = 'UTF-8', sep='^')